In [3]:
# set up libraries
import os
from langchain_openai import ChatOpenAI
openAI_API_KEY = os.getenv("OpenAI_API_KEY")
llm = ChatOpenAI(api_key=openAI_API_KEY)


In [4]:
# test
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated testing: Langsmith can be used to write automated tests for code, ensuring that it functions as expected and does not contain any bugs.\n\n2. Code coverage analysis: Langsmith can analyze code coverage in testing, helping to identify areas of code that are not adequately covered by tests.\n\n3. Performance testing: Langsmith can be used to perform performance testing on code, helping to identify potential bottlenecks and optimize code for better performance.\n\n4. Integration testing: Langsmith can help with integration testing, ensuring that different components of a system work together smoothly.\n\n5. Regression testing: Langsmith can help with regression testing, ensuring that new code changes do not introduce any new bugs or regressions in existing functionality.\n\nOverall, Langsmith can help streamline the testing process and ensure that code is thoroughly tested and of high quality.', response_m

In [5]:
# Guide response with a template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a world class technical documentation writer'),
    ('user', "{input}")
])

In [6]:
# combine into a chain (I think the pipe, "|", is the union set operator. In other scenarios, it could be a bitwise OR operator or a logical OR operator)
chain = prompt | llm

In [7]:
# test the chain
res1 = chain.invoke("how can langsmith help with testing?")


In [8]:
print(res1.response_metadata)
print(res1.content)

{'token_usage': {'completion_tokens': 309, 'prompt_tokens': 27, 'total_tokens': 336}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}
Langsmith is a powerful tool that can greatly assist with testing in various ways. Here are some ways in which Langsmith can help with testing:

1. **Automated Testing**: Langsmith can be used to write automated tests for your codebase. By creating test cases using Langsmith's language generation capabilities, you can quickly generate a large number of test scenarios to validate the functionality of your software.

2. **Boundary Testing**: Langsmith can be used to generate boundary test cases for your code. By specifying the range of inputs and outputs, Langsmith can help identify edge cases that may not be covered by traditional testing methods.

3. **Stress Testing**: Langsmith can be used to simulate stress conditions on your software by generating a large number of concurrent requests or inputs. T

In [9]:
# print out all attributes and their values

for attr_name, attr_val in vars(res1).items():
    print(attr_name, ": ", attr_val)
    print("")


content :  Langsmith is a powerful tool that can greatly assist with testing in various ways. Here are some ways in which Langsmith can help with testing:

1. **Automated Testing**: Langsmith can be used to write automated tests for your codebase. By creating test cases using Langsmith's language generation capabilities, you can quickly generate a large number of test scenarios to validate the functionality of your software.

2. **Boundary Testing**: Langsmith can be used to generate boundary test cases for your code. By specifying the range of inputs and outputs, Langsmith can help identify edge cases that may not be covered by traditional testing methods.

3. **Stress Testing**: Langsmith can be used to simulate stress conditions on your software by generating a large number of concurrent requests or inputs. This can help you identify performance bottlenecks and ensure that your software can handle high loads.

4. **Regression Testing**: Langsmith can generate test cases to ensure th

In [10]:
# use an output parser to make the response more readable
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

In [11]:
# test the new chain
res2 = chain.invoke("how can langsmith help with testing?")

In [12]:
print(res2)

Langsmith is a powerful tool that can greatly assist in testing software applications. Here are some ways in which Langsmith can help with testing:

1. Automated Testing: Langsmith can be used to create automated tests for your software applications. This can help in running tests quickly and efficiently, saving time and effort in manual testing.

2. Test Data Generation: Langsmith can generate test data for your applications, making it easier to test different scenarios and edge cases.

3. Performance Testing: Langsmith can be used to conduct performance testing of your applications, helping to identify bottlenecks and areas of improvement.

4. Integration Testing: Langsmith can help in conducting integration testing of different components of your software applications, ensuring that they work well together.

5. Regression Testing: Langsmith can automate regression testing, helping to ensure that new changes do not introduce any unintended side effects.

Overall, Langsmith can be a v

In [13]:
# retrieval is used when you have too much data to pass to the llm
# So you use a retriever to fetch the most relevant data and pass it to the llm
# Using a web retriever (requires bs4 to be installed)
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()


In [14]:
print(docs)

[Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly un

In [15]:
# Then create embeddings from the docs
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [16]:
# Then build the index (this uses a local vectorstore, faiss-cpu, that needs to be separately installed)
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [17]:
# Now we want to create a retrieval chain that takes a question, looks up relevant documents, and then passes those documents and the original question to the llm

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [18]:
# We want the documents to first come from the retriever we set up
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [19]:
# Test the chain with a question

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications using these datasets, and easily score test results. Developers can upload test cases in bulk, create them on the fly, or export them from application traces. LangSmith also supports custom evaluations (both LLM and heuristic based) to track and diagnose regressions in test scores across multiple revisions of the application.
